# Call an API to get some data

We can call any APIs that are in our Azure 'API Management Instance'.
This is basically a gateway to other APIs. We can add any that meet the OpenAPI Specification.

## PreReqs
- Create an 'API Management Instance'
- Add & publish an API
- These APIs can be open or secured
    - If secured you'll need a subscription key from APIM to connect

## Demo

For secured APIs we'll need to save the subscription key as a `DATABASE SCOPED CREDENTIAL`

1. We'll need a database master key with a proper strong password

In [1]:
IF NOT EXISTS(SELECT * FROM sys.symmetric_keys WHERE name = '##MS_DatabaseMasterKey##')
BEGIN
    create master key encryption by password = '****'
END

Commands completed successfully.

Total execution time: 00:00:00.065

2. Create the `DATABASE SCOPED CREDENTIAL` for the subscription key

In [10]:
DROP DATABASE SCOPED CREDENTIAL [https://pom-api.azure-api.net/conference/speakers]
IF NOT EXISTS (SELECT 1 FROM sys.database_credentials WHERE name = 'https://pom-api.azure-api.net/conference/speakers')
BEGIN
    -- create a database scoped credential for managed identity (or Request Headers or Query String)
    CREATE DATABASE SCOPED CREDENTIAL [https://pom-api.azure-api.net/conference/speakers]
    WITH IDENTITY = 'HTTPEndpointHeaders', SECRET = '{"Ocp-Apim-Subscription-Key":"***"}';
END

Commands completed successfully.

Total execution time: 00:00:00.063

3. Call the API and store the response in a temp table

In [11]:
drop table if exists #temp 

declare @url varchar(500) = 'https://pom-api.azure-api.net/conference/speakers'

declare @ret as int, @response as nvarchar(max);

exec @ret = sp_invoke_external_rest_endpoint
	 @method = 'GET',
	 @url = @url,
     @credential = [https://pom-api.azure-api.net/conference/speakers],
 	 @response = @response output;
	
select @ret as ReturnCode, @response as Response
into #temp

declare @data as nvarchar(max);

select 
	@data = JSON_QUERY(response, '$.result.collection.items')
from #temp;

select 
	ReturnCode, Response
from #temp

(1 row affected)

(1 row affected)

Total execution time: 00:00:01.016

ReturnCode Response 0 {"response":{"status":{"http":{"code":200,"description":"OK"}},"headers":{"Cache-Control":"no-cache","Date":"Sat, 19 Aug 2023 16:29:26 GMT","Pragma":"no-cache","Content-Length":"40606","Content-Type":"application\/vnd.collection+json","Expires":"-1","Set-Cookie":"ARRAffinity=8fc238af3238d4d99f33390b9ab0da11f97e34757d0c5fe645fa69381d77e39d;Path=\/;HttpOnly;Secure;Domain=conferenceapi.azurewebsites.net","Set-Cookie":"ARRAffinitySameSite=8fc238af3238d4d99f33390b9ab0da11f97e34757d0c5fe645fa69381d77e39d;Path=\/;HttpOnly;SameSite=None;Secure;Domain=conferenceapi.azurewebsites.net","X-AspNet-Version":"4.0.30319","X-Powered-By":"ASP.NET"}},"result":{
 "collection": {
 "version": "1.0",
 "href": "https://conferenceapi.azurewebsites.net:443/speakers",
 "links": [],
 "items": [
 {
 "href": "https://conferenceapi.azurewebsites.net/speaker/1",
 "data": [
 {
 "name": "Name",
 "value": "Scott Guthrie"
 }
 ],
 "links": [
 {
 "rel": "http://tavis.net/rels/sessions",
 "href": "https://conferenceapi.azurewebsites.net/speaker/1/sessions"
 }
 ]
 },
 {
 "href": "https://conferenceapi.azurewebsites.net/speaker/2",
 "data": [
 {
 "name": "Name",
 "value": "Don Syme"
 }
 ],
 "links": [
 {
 "rel": "http://tavis.net/rels/sessions",
 "href": "https://conferenceapi.azurewebsites.net/speaker/2/sessions"
 }
 ]
 },
 {
 "href": "https://conferenceapi.azurewebsites.net/speaker/3",
 "data": [
 {
 "name": "Name",
 "value": "James Lewis"
 }
 ],
 "links": [
 {
 "rel": "http://tavis.net/rels/sessions",
 "href": "https://conferenceapi.azurewebsites.net/speaker/3/sessions"
 }
 ]
 },
 {
 "href": "https://conferenceapi.azurewebsites.net/speaker/4",
 "data": [
 {
 "name": "Name",
 "value": "Jessie Shternshus"
 }
 ],
 "links": [
 {
 "rel": "http://tavis.net/rels/sessions",
 "href": "https://conferenceapi.azurewebsites.net/speaker/4/sessions"
 }
 ]
 },
 {
 "href": "https://conferenceapi.azurewebsites.net/speaker/5",
 "data": [
 {
 "name": "Name",
 "value": "Peter Bacon Darwin"
 }
 ],
 "links": [
 {
 "rel": "http://tavis.net/rels/sessions",
 "href": "https://conferenceapi.azurewebsites.net/speaker/5/sessions"
 }
 ]
 },
 {
 "href": "https://conferenceapi.azurewebsites.net/speaker/6",
 "data": [
 {
 "name": "Name",
 "value": "Jon Skeet"
 }
 ],
 "links": [
 {
 "rel": "http://tavis.net/rels/sessions",
 "href": "https://conferenceapi.azurewebsites.net/speaker/6/sessions"
 }
 ]
 },
 {
 "href": "https://conferenceapi.azurewebsites.net/speaker/7",
 "data": [
 {
 "name": "Name",
 "value": "Dominick Baier"
 }
 ],
 "links": [
 {
 "rel": "http://tavis.net/rels/sessions",
 "href": "https://conferenceapi.azurewebsites.net/speaker/7/sessions"
 }
 ]
 },
 {
 "href": "https://conferenceapi.azurewebsites.net/speaker/8",
 "data": [
 {
 "name": "Name",
 "value": "Craig Dunn"
 }
 ],
 "links": [
 {
 "rel": "http://tavis.net/rels/sessions",
 "href": "https://conferenceapi.azurewebsites.net/speaker/8/sessions"
 }
 ]
 },
 {
 "href": "https://conferenceapi.azurewebsites.net/speaker/9",
 "data": [
 {
 "name": "Name",
 "value": "Jez Humble"
 }
 ],
 "links": [
 {
 "rel": "http://tavis.net/rels/sessions",
 "href": "https://conferenceapi.azurewebsites.net/speaker/9/sessions"
 }
 ]
 },
 {
 "href": "https://conferenceapi.azurewebsites.net/speaker/10",
 "data": [
 {
 "name": "Name",
 "value": "Axel Rauschmayer"
 }
 ],
 "links": [
 {
 "rel": "http://tavis.net/rels/sessions",
 "href": "https://conferenceapi.azurewebsites.net/speaker/10/sessions"
 }
 ]
 },
 {
 "href": "https://conferenceapi.azurewebsites.net/speaker/11",
 "data": [
 {
 "name": "Name",
 "value": "Phillip Trelford"
 }
 ],
 "links": [
 {
 "rel": "http://tavis.net/rels/sessions",
 "href": "https://conferenceapi.azurewebsites.net/speaker/11/sessions"
 }
 ]
 },
 {
 "href": "https://conferenceapi.azurewebsites.net/speaker/12",
 "data": [
 {
 "name": "Name",

4. Parse the JSON response with T-SQL
    - then store it in your database or ...

In [12]:
declare @data as nvarchar(max);

select 
	@data = JSON_QUERY(response, '$.result.collection.items')
from #temp;

SELECT [value] as FullName, url
FROM OPENJSON(@data) WITH (
	data2 NVARCHAR(MAX) '$.data' AS JSON,
    url NVARCHAR(50) '$.href'
    )CROSS APPLY OPENJSON(data2)
WITH ([value] NVARCHAR(50))

(99 rows affected)

Total execution time: 00:00:00.049

FullName,url
Scott Guthrie,https://conferenceapi.azurewebsites.net/speaker/1
Don Syme,https://conferenceapi.azurewebsites.net/speaker/2
James Lewis,https://conferenceapi.azurewebsites.net/speaker/3
Jessie Shternshus,https://conferenceapi.azurewebsites.net/speaker/4
Peter Bacon Darwin,https://conferenceapi.azurewebsites.net/speaker/5
Jon Skeet,https://conferenceapi.azurewebsites.net/speaker/6
Dominick Baier,https://conferenceapi.azurewebsites.net/speaker/7
Craig Dunn,https://conferenceapi.azurewebsites.net/speaker/8
Jez Humble,https://conferenceapi.azurewebsites.net/speaker/9
Axel Rauschmayer,https://conferenceapi.azurewebsites.net/speaker/10
